In [1]:
from datasets import load_dataset
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2024-12-26 12:18:41.337439: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shuvoalok/ck-dataset")

print("Path to dataset files:", path)

100%|██████████████████████████████████████| 1.80M/1.80M [00:00<00:00, 19.2MB/s]

Extracting files...


Path to dataset files: /Users/natalyagrokh/.cache/kagglehub/datasets/shuvoalok/ck-dataset/versions/1


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")

print("Path to dataset files:", path)

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Input, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from autogluon.multimodal import MultiModalPredictor

# Load the preprocessed data
X = np.load("preproc_affectnet/X_affectnet.npy")
y = np.load("preproc_affectnet/y_affectnet.npy")

# Reshape the data (if needed) to add a grayscale channel
if len(X.shape) == 3:
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# One-hot encode the labels
num_classes = len(np.unique(y))  # Determine the number of unique classes
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

# Calculate class weights to handle imbalanced datasets
class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y),
    y=y
)
class_weights = dict(enumerate(class_weights))

# Define EfficientNetB0 Model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
model = Sequential([
    tf.keras.layers.Lambda(lambda x: tf.image.grayscale_to_rgb(x)),  # Convert grayscale to RGB
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Define callbacks
callbacks = [
    ModelCheckpoint("final_trained_model.keras", monitor="val_accuracy", save_best_only=True, mode="max"),
    EarlyStopping(monitor="val_loss", patience=5, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1)
]

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,  # Adjust epochs as needed
    batch_size=32,
    class_weight=class_weights,
    callbacks=callbacks
)

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f"Validation Accuracy with EfficientNetB0: {val_accuracy * 100:.2f}%")

# AutoGluon MultiModalPredictor
# Prepare the AutoGluon data using the saved dataset
autogluon_data = pd.DataFrame({
    "image": [Image.fromarray((x.squeeze() * 255).astype(np.uint8), "L").convert("RGB") for x in X],
    "label": y
})

# Initialize and train the AutoGluon predictor
predictor = MultiModalPredictor(label="label")
predictor.fit(autogluon_data, presets="best_quality", time_limit=3600)
print("AutoGluon training completed.")
